# Using machine learning to identity clusters of 'at risk' employees.

In [1]:
## Step 1 - Import necessary libraries
import sklearn
import joblib  # for saving an ml model


Identifying causes of attrition and enabling the business to identify different groups of 'at risk' employees is key. 

This notebook will create a model that will assess the 1400 rows of data currently available (split by 'train' and 'test' groups) to identify if there are logical groupings of exited staff, to enable future departures to be anticipated and, if desired, attempts made to retain.